In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
import pandas as pd
import nltk
from nltk.stem import WordNetLemmatizer

In [92]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

In [91]:
train_df = pd.read_csv('category_train.csv')
test_df = pd.read_csv('category_test.csv')
train_df.head()

,article,category
0,"Either Party shall permit, within the scope o...",5
1,Either Contracting Party shall guarantee with...,5
2,If the present or future legislation of one Co...,14
3,Each Contracting Party shall notify the other ...,11
4,(1) Disputes between contracting parties regar...,8


In [93]:
train_df = train_df[train_df['article'].notnull()]
test_df = test_df[test_df['article'].notnull()]
#lemmatize the text
nltk.download('wordnet')
nltk.download('omw-1.4')
lemmatizer = WordNetLemmatizer()

def lemmatize_text(text):
    return ' '.join([lemmatizer.lemmatize(word) for word in text.split()])

train_df['article'] = train_df['article'].apply(lemmatize_text)
test_df['article'] = test_df['article'].apply(lemmatize_text)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [94]:
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use', '(', ')'])
def remove_stopwords(text):
    return ' '.join([word for word in text.split() if word not in stop_words])

train_df['article'] = train_df['article'].apply(remove_stopwords)
test_df['article'] = test_df['article'].apply(remove_stopwords)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [95]:
tfidf_vectorizer = TfidfVectorizer(stop_words='english',max_features=100)
tfidf_train = tfidf_vectorizer.fit_transform(train_df['article'])
tfidf_test = tfidf_vectorizer.transform(test_df['article'])


In [96]:
Train_Y =train_df['category']
Test_Y = test_df['category']

In [97]:
SVM = svm.SVC(C=1.0, kernel='linear')
SVM.fit(tfidf_train,Train_Y)

SVC(kernel='linear')

In [98]:
predictions_SVM = SVM.predict(tfidf_test)
# Use accuracy_score function to get the accuracy
print("SVM TF-IDF Accuracy Score -> ",accuracy_score(predictions_SVM, Test_Y)*100)
print("SVM TF-IDF F1 Score -> ",f1_score(predictions_SVM, Test_Y, average='weighted')*100)
print("SVM TF-IDF Recall -> ",recall_score(predictions_SVM, Test_Y, average='weighted')*100)
print("SVM TF-IDF precision -> ",precision_score(predictions_SVM, Test_Y, average='weighted')*100)

SVM TF-IDF Accuracy Score ->  91.10852825802702
SVM TF-IDF F1 Score ->  91.70297115029908
SVM TF-IDF Recall ->  91.10852825802702
SVM TF-IDF precision ->  92.74505604769101


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [103]:
Naive = naive_bayes.MultinomialNB()
Naive.fit(tfidf_train,Train_Y)
# predict the labels on validation dataset
predictions_NB = Naive.predict(tfidf_test)
# Use accuracy_score function to get the accuracy
print("Naive Bayes TF-IDF Accuracy Score -> ",accuracy_score(predictions_NB, Test_Y)*100)
print("Naive Bayes TF-IDF F1 Score -> ",f1_score(predictions_NB, Test_Y, average='weighted')*100)
print("Naive Bayes TF-IDF Recall -> ",recall_score(predictions_NB, Test_Y, average='weighted')*100)
print("Naive Bayes TF-IDF precision -> ",precision_score(predictions_NB, Test_Y, average='weighted')*100)

Naive Bayes TF-IDF Accuracy Score ->  80.95307278802848
Naive Bayes TF-IDF F1 Score ->  84.93069702909327
Naive Bayes TF-IDF Recall ->  80.95307278802848
Naive Bayes TF-IDF precision ->  91.15879863132108


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [104]:
import gensim
import gensim.downloader as gensim_api
from gensim.models import Word2Vec
import numpy

In [105]:


sent_train = [row.split(' ') for row in train_df['article']]
sent_test = [row.split(' ') for row in test_df['article']]

sent= sent_train + sent_test


In [87]:
model = Word2Vec(sent, min_count=1,size= 300,workers=3, window =3, sg = 1)

In [109]:
word_vec_train = [numpy.average(model[row.split(' ')],axis = 0) for row in train_df['article']]
word_vec_test = [numpy.average(model[row.split(' ')],axis = 0) for row in test_df['article']]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [107]:
len(word_vec_train[0])

300

In [110]:
SVM_wv = svm.SVC(C=1.0, kernel='rbf')
SVM_wv.fit(word_vec_train,Train_Y)
predictions_SVM_wv = SVM_wv.predict(word_vec_test)
# Use accuracy_score function to get the accuracy
print("SVM WV Accuracy Score -> ",accuracy_score(predictions_SVM_wv, Test_Y)*100)
print("SVM WV F1 Score -> ",f1_score(predictions_SVM_wv, Test_Y, average='weighted')*100)
print("SVM WV Recall -> ",recall_score(predictions_SVM_wv, Test_Y, average='weighted')*100)
print("SVM WV precision -> ",precision_score(predictions_SVM_wv, Test_Y, average='weighted')*100)

SVM WV Accuracy Score ->  92.00929827110271
SVM WV F1 Score ->  92.7901593189673
SVM WV Recall ->  92.00929827110271
SVM WV precision ->  94.12857793330947


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
